In [10]:
import gradio as gr
from transformers import pipeline,BartForConditionalGeneration, AutoTokenizer

In [11]:
loaded_model = BartForConditionalGeneration.from_pretrained("ManuVleuBeu/BART_answer-aware_MSMARCO").to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained("ManuVleuBeu/BART_answer-aware_MSMARCO")

In [12]:
def run_model(input_string,target_answer, **generator_args):
    generator_args = {
    "max_length": 768,
    "num_beams": 4,# bij grotere num_beams is trager maar complexere vragen(niet per se betere vragen)
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    
    input_string = input_string + " " + "<ANSWER>" + target_answer + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = loaded_model.generate(input_ids.to("cuda:0"), **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    output = [item.split("<sep>") for item in output]
    return output

In [ ]:
# question = "Who is Fyodor Dostoevsky?"
#TODO: add a select chapter from GF context
#TODO: get context 
context = "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century."

pipe_qg=pipeline("text2text-generation",model="ManuVleuBeu/BART_answer-aware_MSMARCO")
pipe_qa = pipeline("question-answering", model="damapika/roberta-base_mod")

def getQuestion(context):
  print(pipe_qg(context=context))
  
def getAnswer(question, context):
  return pipe_qa(question=question, context=context)['answer']

getQuestion(context)

demo = gr.Interface(
  fn=getAnswer, 
  inputs=['text','text'],
  outputs='text',
)

demo.launch()

In [4]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

with gr.Blocks() as demo:
    question = gr.Textbox(label="Question")
    answer = gr.Textbox(label="Answer")
    ans_btn = gr.Button("Check answer")
    ans_btn.click(fn=greet, inputs=question, outputs=answer, api_name="greet")


demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
